# Transcription Experiment

This notebook demonstrates the choice of transcriber used for the project considering the resources at hand.


## Decision
1. Framework : **OpenAI Whisper** 
2. Model : **hybrid method** (medium model with GPU acceleration / base model with CPU fallback)

### Step0: Framework Choice
    
1. OpenAI Whisper
    
    Pros: 
        
    - open source
    - high accuracy (multi-languages, long conversation)
    - support CPU/GPU
    - full pipeline (can output transcript without processing)
        
    Cons: 

    - large models are computational expensive
    - limited capabilities compared to commercial APIs

2. HuggingFace Whisper
    - built on OpenAI Whisper, adds extra dependency without major accuracy improvements

3. Assembly AI
    - not considered here since it requires paid commercial API keys


**Framework Conclusion : OpenAI Whisper**



### Step1 : Env and Device Setup 

In [1]:
# !pip install git+https://github.com/openai/whisper.git
# !pip install torch torchaudio
# !pip install ffmpeg-python  # Python wrapper for ffmpeg

import os
import whisper
import torch
import time
import psutil
from pathlib import Path
import pandas as pd
import numpy as np

In [30]:
AUDIO_PATH = Path("../sample_data/sample_audio.wav")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print(f"Using device: {DEVICE}")

Using device: cpu


### Step2 : Define a custom function for transcription

In [31]:
def get_memory_usage_mb():
    """Return current memory usage (MB) of this process."""
    process = psutil.Process(os.getpid())
    return process.memory_info().rss / (1024 * 1024)

In [32]:
def transcribe_audio(audio_path, model_size, device=DEVICE):
    """
    Transcribe audio using OpenAI Whisper
    
    Args:
        audio_path (str or Path): path to audio file
        model_size (str): tiny, base, small, medium, large
        device (str): "cpu" or "cuda"
    
    Returns:
        transcript (str)
        elapsed_time (float)
        peak_memory_mb (float)
    """
    # record memory before and after loading model
    mem_before = get_memory_usage_mb()
    
    model = whisper.load_model(model_size, device=device)
    
    mem_after_load = get_memory_usage_mb()

    start_time = time.time()
    result = model.transcribe(str(audio_path))
    elapsed_time = time.time() - start_time

    # record memory after transcription to calculate peak memory
    mem_after_transcribe = get_memory_usage_mb()

    transcript = result["text"]

    peak_memory_mb = max(mem_before, mem_after_load, mem_after_transcribe)

    print(f"Transcription done in {elapsed_time:.2f} sec, "
          f"Peak Memory: {peak_memory_mb:.1f} MB")

    return transcript, elapsed_time, peak_memory_mb

### Step3: Try Different Model Size

In [35]:
models_to_test = ["tiny", "base", "small", "medium"]  # large might be too much for CPU (expected 5 GB of memory)

results = {}

for model_name in models_to_test:
    print(f"\n=== Model: {model_name} ===")
    transcript, elapsed, peak_memory_mb = transcribe_audio(AUDIO_PATH, model_size=model_name, device=DEVICE)
    results[model_name] = {
        "transcript": transcript,
        "time_sec": elapsed,
        "peak_memory_mb": peak_memory_mb
    }
    print("Preview:\n" + transcript[:200])  # preview the first 200 characters


=== Model: tiny ===


/Users/hansonshen/Python/ml-takehome/venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription done in 9.89 sec, Peak Memory: 696.8 MB
Preview:
 So let's take it past the point where you have the scales, you have reusable ship. Yeah. You've got it dialed in, then what are the steps? What's next step after that? Is it an unmanned voyage tomorr

=== Model: base ===


/Users/hansonshen/Python/ml-takehome/venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription done in 18.33 sec, Peak Memory: 913.5 MB
Preview:
 So let's let's take it past the point where you have these scales you have a reusable ship Yeah, and you've you've got it dialed in then what are the steps? What what's next step after that is it an 

=== Model: small ===


/Users/hansonshen/Python/ml-takehome/venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription done in 45.26 sec, Peak Memory: 1528.1 MB
Preview:
 So let's let's take it past the point where you have these scales you have a reusable ship Yeah, and you've got it dialed in then what are the steps? What what's next step after that? Is it an unmann

=== Model: medium ===


/Users/hansonshen/Python/ml-takehome/venv/lib/python3.12/site-packages/whisper/transcribe.py:132: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


Transcription done in 142.29 sec, Peak Memory: 4594.5 MB
Preview:
 So let's let's take it past the point where you have these scales you have a reusable Ship yeah, and you've got it dialed in then. What are the steps? What what's next step after that is it an unmann


### Step4 : Compare the Results

In [ ]:
# compare the time and memory usage of different models
summary_df = pd.DataFrame([
    {"model": m, 
     "time_sec": results[m]["time_sec"], 
     "peak_memory_mb": results[m]["peak_memory_mb"], 
     "transcript_preview": results[m]["transcript"][:80]}
    for m in results
])

summary_df

,model,time_sec,peak_memory_mb,transcript_preview
0,tiny,9.892284,696.750000,So let's take it past the point where you hav...
1,base,18.327907,913.500000,So let's let's take it past the point where y...
2,small,45.264171,1528.140625,So let's let's take it past the point where y...
3,medium,142.289918,4594.484375,So let's let's take it past the point where y...


In [40]:
# show all results and compare the the full content of the transcript
for m in results:
    print(f"Model: {m}")
    print(f"Transcript: {results[m]['transcript']}")
    print("\n")

Model: tiny
Transcript:  So let's take it past the point where you have the scales, you have reusable ship. Yeah. You've got it dialed in, then what are the steps? What's next step after that? Is it an unmanned voyage tomorrow's first? I'm Anne Fliedamores. The Earth and Mars will be synchronized every two years. Or every 26 months, technically. So the next orbital synchronization is November of next year. So, and you can launch plus minus a month roughly. So we'd have to launch in November or December of next year. And so the default plan is to launch hopefully several starships tomorrow at the end of next year. And what would they be doing? Well, at first we're just going to try to land on Mars and see if we succeed in landing. Do we do we succeed in landing? Like let's say we were able to send five ships to all five land intact or do we add some creators to Mars? If we add some creators, we've got to be put more courses about setting people. You know, we need to. So we're going to m

### Observation (3 aspects to consider)

1.	Execution Performance (Latency & Memory)
- Tiny and base model sizes are CPU-friendly and good for fast prototyping, with execution time < 20s and memory < 1024MB
- Small and medium models take much longer (45/142s) and requires substantially more memory (1.5/4.6 GB)
- Trade-off: larger models significantly increase both runtime and memory usage and therefore require to be run on GPU or cloud for practical development

2.	Transcription Quality (Accuracy & Robustness)
- Tiny： "So let’s take it ..." → an understanable prototype but slightly less consistent
- Base/Small/Medium： "So let’s let’s take ..." → better capture the repetitive words the speaker said, showing improved fluency and stability
- The medium model is expected to yield the best accuracy overall, though at a very high computational cost.

3.	Task Requirements (Practical Fit and Pipeline Design)
- If the goal is fast prototyping or CPU-only environments, tiny or base strikes the best balance
- If accuracy is critical and GPU resources are available, small or medium may be justified despite the extra cost
- Since the next step in the pipeline is Topic Extraction, the "perfect" quality of the transcription is not the priority; instead, the ability to capture most of the content is


### Trade-off and Conclusion

For the pipeline, a hybrid method is used. (DEVICE = "cuda" if torch.cuda.is_available() else "cpu")
- **Base** is chosen as the default model, offering the best trade-off between speed and accuracy on CPU. 
- **Medium** is used when GPU resources are available, ensuring the system remains scalable and hardware-aware.